In [1]:
import tensorflow as tf
import joblib


def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

cfg = tf.ConfigProto(allow_soft_placement=True )
cfg.gpu_options.allow_growth = True

from tensorflow.layers import batch_normalization

c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shubh\appd

In [2]:
data = joblib.load("mnist_dataset.pkl")

In [3]:
data_X = data.data
data_y = data.target

import numpy as np
def shuffle(data_x , data_y , test_ratio = 0.1):
    shuffle_indices = np.random.permutation(data_x.shape[0])
    test_indices = shuffle_indices[:int(test_ratio*len(shuffle_indices))]
    train_indices = shuffle_indices[int(test_ratio*len(shuffle_indices)):]
    train_x , train_y ,test_x , test_y = data_x[train_indices] , data_y[train_indices] , data_x[test_indices] ,data_y[test_indices]
    return train_x , train_y , test_x ,test_y

In [4]:
train_x , train_y ,test_x , test_y = shuffle(data_X , data_y)
valid_x = test_x[:len(test_x)//2]
valid_y = test_y[:len(test_y)//2]

In [5]:
n_inputs = 784
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [6]:
reset_graph()
x = tf.placeholder(tf.float32 ,shape= (None , n_inputs) , name = "x")
y = tf.placeholder(tf.int32 , shape = (None ,) , name ="y") 

training = tf.placeholder_with_default(False, shape=(), name='training')
y.shape

TensorShape([Dimension(None)])

In [7]:
reset_graph()
x = tf.placeholder(tf.float32 ,shape= (None , n_inputs) , name = "x")
y = tf.placeholder(tf.int32 , shape = (None , ) , name ="y") 

training = tf.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope("hidden_layers"):
    hid_1 = tf.layers.dense(x , n_hidden1 , name = "hidden1")
    batch_n1 = batch_normalization(hid_1 , training= training , momentum=0.9)
    batch_n1_act = tf.nn.elu(batch_n1)
    
    hid_2 = tf.layers.dense(batch_n1_act , n_hidden2 , name = "hidden2")
    batch_n2 = batch_normalization(hid_2 , training= training , momentum=0.9)
    batch_n2_act = tf.nn.elu(batch_n2)
 
    output = tf.layers.dense(batch_n2_act  , n_outputs , name = "output")
    final_out = batch_normalization(output , training= training , momentum=0.9)
    
final_out.shape  

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


TensorShape([Dimension(None), Dimension(10)])

In [8]:
# from functools import partial

# my_batch_norm_layer = partial(tf.layers.batch_normalization,
#                               training=training, momentum=0.9)

# hidden1 = tf.layers.dense(x, n_hidden1, name="hidden1")
# bn1 = my_batch_norm_layer(hidden1)
# bn1_act = tf.nn.elu(bn1)
# hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
# bn2 = my_batch_norm_layer(hidden2)
# bn2_act = tf.nn.elu(bn2)
# logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
# logits = my_batch_norm_layer(logits_before_bn)

In [9]:
with tf.name_scope("loss"):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=final_out)
    loss = tf.reduce_mean(cross_entropy)

In [10]:
with tf.name_scope("training"):
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    training_op = optimizer.minimize(loss)

In [11]:
with tf.name_scope("metrics"):
    correct = tf.nn.in_top_k(final_out , y , 1)
    accuracy = tf.reduce_mean(tf.cast(correct , tf.float32))

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
def shuffled_batch(x , y , shuffled=True ,batch_size = 100):
    if shuffled:
        n_batches = len(x)//batch_size
        for batch_idx in np.array_split(np.arange(len(x)) , n_batches):
            x_batch , y_batch = x[batch_idx] , y[batch_idx]
            yield x_batch , y_batch
    else:
        
        rnd_idx = np.random.permutation(len(x))
        n_batches = len(x) // batch_size
        for batch_idx in np.array_split(rnd_idx, n_batches):
            x_batch, y_batch = x[batch_idx], y[batch_idx]
            yield x_batch, y_batch

In [14]:
n_epochs = 20
batch_size = 200
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session(config=tf.ConfigProto(
  allow_soft_placement=True, log_device_placement=True)) as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffled_batch(train_x , train_y, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, x: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={x: valid_x , y:valid_y})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./myfinal.ckpt")

0 Validation accuracy: 0.90314287
1 Validation accuracy: 0.9277143
2 Validation accuracy: 0.9434286
3 Validation accuracy: 0.9522857
4 Validation accuracy: 0.958
5 Validation accuracy: 0.9605714
6 Validation accuracy: 0.96428573
7 Validation accuracy: 0.9671429
8 Validation accuracy: 0.97
9 Validation accuracy: 0.97171426
10 Validation accuracy: 0.97257143
11 Validation accuracy: 0.974
12 Validation accuracy: 0.974
13 Validation accuracy: 0.97514284
14 Validation accuracy: 0.9754286
15 Validation accuracy: 0.97514284
16 Validation accuracy: 0.97514284
17 Validation accuracy: 0.97514284
18 Validation accuracy: 0.97657144
19 Validation accuracy: 0.9771429
